In [1]:
%run CNNtf.ipynb
%run config.ipynb
%run board_object.ipynb
import sys 
from collections import OrderedDict
import os
import random
import dill as pickle

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [3]:
#configs: learning rate, batch size, batch size2, epochs

In [ ]:
tf.reset_default_graph()

In [5]:
def policy_move(board_obj_obj, network1, params1_v):
    turn = board_obj_obj.turn
    

    
    
    if turn == 1:

        #argmax needs to be correctly implemented
        #print(loop_model_scores([reformat2(board_obj_obj)], True,  network1, params1_v)[0])
        i = [prop_choice(loop_model_scores([reformat2(board_obj_obj)], True,  network1, params1_v)[0])]
        #i = np.argmax(loop_model_scores([reformat2(board_obj_obj)], True,  network1, params1_v), axis = 1)
        print(i)
        board_obj_obj.move(i)
    elif turn == 2:
        
        i = [prop_choice(loop_model_scores([switch_reformat(reformat2(board_obj_obj))], True,  network1, params1_v)[0])]
        #i = np.argmax(loop_model_scores([switch_reformat(reformat2(board_obj_obj))], True,  network1, params1_v), axis = 1)
        print(i)
        board_obj_obj.move(i)
    return (board_obj_obj, i)

In [ ]:
"""WRONG need to somehow keep policy CNN for generation, unless we do them together"""


def policy_move_val(board_obj_obj, network1, params1_v):
    turn = board_obj_obj.turn
    

    
    
    if turn == 1:

        #argmax needs to be correctly implemented
        #print(loop_model_scores([reformat2(board_obj_obj)], True,  network1, params1_v)[0])
        i = [prop_choice(loop_model_scores_val([reformat2(board_obj_obj)], True,  network1, params1_v)[0])]
        #i = np.argmax(loop_model_scores([reformat2(board_obj_obj)], True,  network1, params1_v), axis = 1)
        print(i)
        board_obj_obj.move(i)
    elif turn == 2:
        
        i = [prop_choice(loop_model_scores_val([switch_reformat(reformat2(board_obj_obj))], True,  network1, params1_v)[0])]
        #i = np.argmax(loop_model_scores([switch_reformat(reformat2(board_obj_obj))], True,  network1, params1_v), axis = 1)
        print(i)
        board_obj_obj.move(i)
    return (board_obj_obj, i)

In [6]:
def sample_trajectory(network1, params1_v):
    
    board_obj_obj = board_obj()
    states = [board_obj_obj]
    target = []
    board_obj_obj = board_obj_obj.copy()
    while evaluate(board_obj_obj.board) not in [0, 0.5, 1]:
        p = policy_move(board_obj_obj, network1, params1_v)
        (board_obj_obj, i) = (p[0].copy(), p[1]) 
        states.append(board_obj_obj)
        target.append(i)
    outcome = evaluate(board_obj_obj.board)
    
    
    return [states, outcome, target]
    
    

In [ ]:
def sample_trajectory_val(network1, params1_v, network_policy, params_policy):
    
    
    
    return sample_trajectory(network_policy, params_policy)
    

In [7]:
def iteration(params1_v, batch_size, network1):
    #print(params1_v)
    params1_va = params1_v.copy()
    for count in range(proper_batch_size):
        #print(count)
        [states, outcome, target] = sample_trajectory(network1, params1_va)


        


        for i in range(len(target)):
            turn_cur = states[i].turn
            #make sure the cnn_action() network in cnn_action_grad function definition body is IMMUTABLE
            if turn_cur == 1:
                grad = loop_model_grad([reformat2(states[i])], target[i], True, network1, params1_va)
                #grad = np.array(grad)
            elif turn_cur == 2:
                grad = loop_model_grad([switch_reformat(reformat2(states[i]))], target[i], True, network1, params1_va)
                #grad = np.array(grad)
            else:
                print('Turn Error')
                sys.exit()

            if turn_cur == 1:
                #for param, grad_ind in zip(params1, grad):
                    #updates1[param] = param + learning_rate * grad_ind * outcome
                grad = np.array(grad)
                params1_va = np.array(params1_va)
                params1_va = params1_va - learning_rate * grad * outcome    
            elif turn_cur == 2:
                #for param, grad_ind in zip(params2, grad):
                    #updates2[param] = param + learning_rate * grad_ind * outcome
                grad = np.array(grad)
                params1_va = np.array(params1_va)
                params1_va = params1_va - learning_rate * grad * (1 - outcome)     
            else:
                print('Turn Error')
                sys.exit()
                
               
        print('one_loop_done')
    print(params1_va[0][0][0])
    print('break')
    print(params1_v[0][0][0])
    return params1_va




      
def iteration_action(params1_v, batch_size, network1):
    #print(params1_v)
    params1_va = params1_v.copy()
    
    
    #need to add something that sets graph to pol graph
    pipe = test_RESIDUAL_net_action_1([1])
    
    network_pol = RESIDUAL_net_action_1
    params_pol = loop_model_params_get(True, network_pol)
    for count in range(proper_batch_size):
        #print(count)
        [states, outcome, target] = sample_trajectory(network_pol, params_pol)


        T = len(target)-1
        i = random.choice(list(range(T)))
        choice_2 = random.choice(list(range(T)))
        
        
        turn_cur = states[i].turn
        
        
        #make sure the cnn_action() network in cnn_action_grad function definition body is IMMUTABLE
        if turn_cur == 1:
            loss, grad = loop_model_grad_alt([reformat2(states[i])], True, network1, params1_va)
            #grad = np.array(grad)
        elif turn_cur == 2:
            loss, grad = loop_model_grad_alt([switch_reformat(reformat2(states[i]))], True, network1, params1_va)
            #grad = np.array(grad)
        else:
            print('Turn Error')
            sys.exit()

        
        ###Need to fix gradient update
        
        
        
        if turn_cur == 1:
            #for param, grad_ind in zip(params1, grad):
                #updates1[param] = param + learning_rate * grad_ind * outcome
            grad = np.array(grad)
            params1_va = np.array(params1_va)
            params1_va = params1_va + learning_rate * grad * (outcome - loss)    
        elif turn_cur == 2:
            #for param, grad_ind in zip(params2, grad):
                #updates2[param] = param + learning_rate * grad_ind * outcome
            grad = np.array(grad)
            params1_va = np.array(params1_va)
            params1_va = params1_va + learning_rate * grad * (1 - outcome - loss)     
        else:
            print('Turn Error')
            sys.exit()

               
        print('one_loop_done')
    print(params1_va[0][0][0])
    print('break')
    print(params1_v[0][0][0])
    return params1_va
    
    

In [ ]:
def REINFORCEloop(epochs):
    network1 = RESIDUAL_net_action_1
    pipe = test_RESIDUAL_net_action_1([1])
    for i in range(epochs):
        #print(i)
        params1 = loop_model_params_get(True, network1)
        u = iteration(params1, batch_size, network1).copy()
        loop_model_params_set(True, network1, u)
        
        
    return network1


def Valueloop(epochs):
    network1 = RESIDUAL_net_value_1
    pipe = test_RESIDUAL_net_value_1([1])
    for i in range(epochs):
        #print(i)
        params1 = loop_model_params_get(True, network1)
        u = iteration_action(params1, batch_size, network1).copy()
        loop_model_params_set(True, network1, u)
        
        
    return network1

def REINFORCEloop2(epochs):
    network1 = RESIDUAL_net_action_1
    pipe = test_RESIDUAL_net_action_1([1])
    for i in range(epochs):
        #print(i)
        params1 = loop_model_params_get(True, network1)
        u = iteration(params1, batch_size, network1).copy()
        loop_model_params_set(True, network1, u)
    
    
    
    with open('policy_model.pkl', 'wb') as file:
        pickle.dump(params1, file)

    with open('policy_model.pkl', 'rb') as file:
        params1 = pickle.load(file)
        
    return network1


def Valueloop2(epochs):
    network1 = RESIDUAL_net_value_1
    pipe = test_RESIDUAL_net_value_1([1])
    for i in range(epochs):
        #print(i)
        params1 = loop_model_params_get(True, network1)
        u = iteration_action(params1, batch_size, network1).copy()
        loop_model_params_set(True, network1, u)
    
    
    with open('value_model.pkl', 'wb') as file:
        pickle.dump(params1, file)

    with open('value_model.pkl', 'rb') as file:
        params1 = pickle.load(file)
        
    return network1